# Hyperparameter Tuning using HyperDrive

### Import Dependencies
In the cell below, importing all the dependencies that will be needed to complete the project.

In [1]:
from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive import normal, uniform, choice
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

### Overview

Attrition has always been a major concern in any organization. The IBM HR Attrition Case Study is a fictional dataset which aims to identify important factors that might be influential in determining which employee might leave the firm and who may not.

Dataset link: https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset

The Dataset consists of 35 columns, through which we aim to predict weather an employee will leave the job or not. This is a binary classification problem, where the outcome 'Attrition' will either be '0' or '1'. 
In this experiment we will be using HyperDrive and Decision Tree to optimise the best model for the given Dataset.

### Import Workspace

In [2]:
ws = Workspace.from_config()

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id,
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-138608
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-138608


### Create Experiment

In [3]:
# choosing a name for experiment

experiment_name = 'capstone-hyperdrive'
experiment=Experiment(ws, experiment_name)
 
run = experiment.start_logging()

### Create Compute Cluster

In [4]:
cluster_name = "notebook138608"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target, using it!')
except ComputeTargetException:
    print('Creating a new compute target!')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    
    # create the cluster
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)
 
# Using get_status() to get a detailed status for the current cluster.
print(cpu_cluster.get_status().serialize())

Found existing compute target, using it!

Running
{'errors': [], 'creationTime': '2021-02-11T17:50:41.294287+00:00', 'createdBy': {'userObjectId': '8528587c-835e-4446-87fb-ee3ba202662a', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': None}, 'modifiedTime': '2021-02-11T17:53:14.698308+00:00', 'state': 'Running', 'vmSize': 'STANDARD_DS3_V2'}


## Dataset

In the cell below, writing code to access the external data that will be used in this project. We are using the IBM HR Analytics Employee Attrition & Performance from Kaggle

In [5]:
# Loading the dataset from the Workspace. Otherwise, creating it from the file.
found = False
key = "Employee Attrition"
description_text = "IBM HR Analytics Employee Attrition & Performance"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        data = 'https://raw.githubusercontent.com/manas-v/Capstone-Project-Azure-Machine-Learning-Engineer/main/WA_Fn-UseC_-HR-Employee-Attrition.csv'
        dataset = Dataset.Tabular.from_delimited_files(data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,...,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.192517,2.912925,1.0,1024.865306,2.721769,65.891156,2.729932,2.063946,...,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.106864,1.024165,0.0,602.024335,1.093082,20.329428,0.711561,1.106940,...,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,1.0,491.250000,2.000000,48.000000,2.000000,1.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1.0,1020.500000,3.000000,66.000000,3.000000,2.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,4.000000,1.0,1555.750000,4.000000,83.750000,3.000000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,2068.000000,4.000000,100.000000,4.000000,5.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


## Hyperdrive Configuration

The HyperDrive configuration here includes data about hyperparameter space. 

We have used RandomParameterSampling as the Sampling method, BanditPolicy as the Termination policy, SKLearn estimator with the Primary metric as AUC_weighted.
(More details in the readme file)

In [6]:
# Creating an early termination policy.
early_termination_policy = BanditPolicy(slack_factor=0.1,evaluation_interval=3)

#Creating the different params that will be used during training
param_sampling = RandomParameterSampling({"--criterion": choice("gini", "entropy"),"--splitter": choice("best", "random"), "--max_depth": choice(3,4,5,6,7,8,9,10)})

#Creating estimator and hyperdrive config
estimator = SKLearn(source_directory=".", compute_target=cpu_cluster, entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                         policy=early_termination_policy, 
                                         primary_metric_name="AUC_weighted", 
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                         max_total_runs=12, 
                                         max_concurrent_runs=4, 
                                         estimator=estimator)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [7]:
#Submitting the experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

Using the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

Getting the best model from the hyperdrive experiments and displaying all the properties of the model.

In [9]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
 
print('Best Run Id: ', best_run.id)
print('AUC_weighted of Best Run is:', best_run_metrics['AUC_weighted'])
print('Parameter Values are:',best_run.get_details()['runDefinition']['arguments'])

Best Run Id:  HD_6462ea46-7e5e-4054-8a2d-a4dea8d17f48_5
AUC_weighted of Best Run is: 0.7214713617767388
Parameter Values are: ['--criterion', 'gini', '--max_depth', '4', '--splitter', 'best']


In [10]:
#Saving the best model
model = best_run.register_model(model_name='hyperdrive-model', 
                                model_path='outputs/model.joblib', 
                                tags={'Method':'Hyperdrive'}, 
                                properties={'AUC_weighted': best_run_metrics['AUC_weighted']})